In [ ]:
# got it working but predictions are far off, I'll need to experiment with architecture.
# it may be the case of whole lines being too much for this type of algorithm
# and I need to give it bar-by-bar treatment first,
# and then augument that.

# as far as my research goes it's a rather unexplored territory to try and translate
# sheet music into text

In [1]:
from numpy.random import normal
from random import choice
from random import randrange, random
import shutil as s
import os
import pandas as pd

class Note:
    
    def __init__(self, start = 0, octave = 0):
        self.scale = ['c','d','e','f','g','a','b']
        self.current = start
        self.octave = octave
        
    def to_string(self):
        note = self.scale[self.current]
        octave_mark = ''
        if self.octave > 0:
            octave_mark = "'" * self.octave
        if self.octave < 0:
            octave_mark = "," * abs(self.octave)
        return str(self.scale[self.current]+octave_mark)
    
    def __add__(self, other):
        return self.jump(other)
        
    def __sub__(self, other):
        return self.jump(-1*other)
        
    def jump(self, steps):
        self.current += steps
        self.octave += self.current//7
        self.current %= 7
        return Note(self.current, self.octave)
        
    def relative(self, relative_to):
        return self.current + relative_to * 7
    
    def absolute(self):
        return self.current+self.octave*7
        
class Instrument:
    def __init__(self, bot = -3, top = 30, name = 'violin'):
        self.variance = 2
        self.bottom_border = bot
        self.up_border = top
        self.start_octave = 1
        self.current = Note(octave = self.start_octave)
        self.jumps = []
        self.notes = []
        self.pages = 0
        
    def gen_next(self, offset = False):
        if not offset:
            offset = round(normal(loc=-2.0, scale=self.variance))
        
        while offset == 0: # take random offset until value is different from 0 - we don't want multiple notes one after another with the same pitch
            offset = round(normal(loc=-2.0, scale=self.variance))
        
        # print(f"{self.current.to_string()}: {self.current.absolute()}, {offset}")
        
        while self.current.absolute() + offset < self.bottom_border: #tu jest coś nie tak bo nie ogranicza ani w górę ani w dół
            #print("{} is too low, jumping by {}".format(offset, offset+6))
            offset += 9
        while self.current.absolute() + offset > self.up_border:
            #print("{} is too high, jumping by {}".format(offset, offset-6))
            offset -= 9
            
        return self.current + offset, offset
    
    def _gen_one(self, offset = False):
        if not offset:
            offset = round(normal(loc=-2.0, scale=self.variance))
        
        while offset == 0: # take random offset until value is different from 0 - we don't want multiple notes one after another with the same pitch
            offset = round(normal(loc=-2.0, scale=self.variance))
        
        print(f"{self.current.to_string()}: {self.current.absolute()}, {offset}")
        
        while self.current.absolute() + offset < self.bottom_border: #tu jest coś nie tak bo nie ogranicza ani w górę ani w dół
            #print("{} is too low, jumping by {}".format(offset, offset+6))
            offset += 6
        while self.current.absolute() + offset > self.up_border:
            #print("{} is too high, jumping by {}".format(offset, offset-6))
            offset -= 6
    
        self.current = self.current + offset
        
        self.jumps.append(offset)
        self.notes.append(self.current)
    
    def gen_n(self, n):
        for i in range(n):
            if i>0:
                if i%4==0:
                    self.notes.append("|")
                if i%32==0:
                    self.notes.append("\pageBreak")
                    self.pages += 1
            self.notes.append(self.current)
            self.current = self.gen_next()[0]
        self.notes.append("|")
        self.notes.append("\pageBreak")
        self.pages += 1
                
    def to_file(self, name = 'test.ly'):
        strings = [x.to_string() if isinstance(x, Note) else x for x in self.notes]
        
        file = open(name,'w')
        file.write("""
#(set! paper-alist
  (cons '("my size" . (cons (* 21 cm) (* 2 cm))) paper-alist))

\paper {
  #(set-paper-size "my size")
  print-page-number=false
}
        
\layout {
    indent = 0\cm
    \context {
        \Score
        \omit BarNumber
    }
}
{ \\fixed c'{\hide TimeSignature\n""")
        file.write(' '.join(strings))
        file.write("\n}}")
        file.close()
        return strings
        
    def make_images(self, name='test.ly'):
        strings = self.to_file(name)
        !"C:\Program Files (x86)\LilyPond\usr\bin\lilypond.exe" -fpng test.ly
        return strings
        
    def make_csv(self, name="test.csv"):
        x = [[] for a in range(self.pages)]
        counter = 0
        for i, j in enumerate(self.notes):
            if counter==self.pages:
                break
            if j == "\pageBreak":
                counter += 1
            if j != "\pageBreak":
                if isinstance(j, Note):
                    x[counter].append(j.to_string())
                else:
                    x[counter].append(j)
                
        data = pd.DataFrame(x)
        data["path"] = [f"{i+1}.png" for i in range(data.shape[0])]
        data.to_csv(name)
        
        return data
        
    def generate_dataset_and_labels_file(self, direc="test"):
        """
        it was a long day.
        """
        self.make_csv()
        
        for file in os.listdir():
            if file.endswith(".png"):
                #pages += 1
                s.move(file, "test")
                old = os.path.join("test", file)
                new = os.path.join("test", file.replace("test-page", ""))
                os.rename(old, new)
            if file.endswith(".csv"):
                s.move(file, "test")
        

In [2]:
a = Instrument()

In [3]:
a.gen_n(4000)
strings = a.make_images()
dictionary = set(strings)

Processing `test.ly'
Parsing...
test.ly:1: warning: no \version statement found, please add

\version "2.22.1"

for future compatibility
Interpreting music...[8][16][24][32][40][48][56][64][72][80][88][96][104][112][120][128][136][144][152][160][168][176][184][192][200][208][216][224][232][240][248][256][264][272][280][288][296][304][312][320][328][336][344][352][360][368][376][384][392][400][408][416][424][432][440][448][456][464][472][480][488][496][504][512][520][528][536][544][552][560][568][576][584][592][600][608][616][624][632][640][648][656][664][672][680][688][696][704][712][720][728][736][744][752][760][768][776][784][792][800][808][816][824][832][840][848][856][864][872][880][888][896][904][912][920][928][936][944][952][960][968][976][984][992][1000]
Preprocessing graphical objects...
Finding the ideal number of pages...
Fitting music on 124 or 125 pages...
Drawing systems...
Converting to PNG...
Success: compilation successfully completed


In [4]:
dictionary

{'\\pageBreak',
 'a',
 "a'",
 'a,',
 'b',
 'b,',
 'c',
 "c'",
 "c''",
 'd',
 "d'",
 'e',
 "e'",
 'f',
 "f'",
 'g',
 "g'",
 'g,',
 '|'}

In [26]:
try:
    dictionary.remove('\\pageBreak')
except KeyError:
    pass
dictionary.add('x')

In [27]:
a.generate_dataset_and_labels_file()

Error: Destination path 'test\test.csv' already exists

In [7]:
# Losowy rytm - szkic
# -losowo przydziel nutom wartości z predefiniowanych grup (4 ósemki, 2 ósemki, ćwierćnuta, ćwierćnuta z kropką + ósemka, półnuta itd)
# -idąc od początku powkładaj wartości do taktów
# -utnij to co zostało do pełnego taktu
# -yeeepeee mamy losową melodię

In [8]:
from tensorflow.keras.layers.experimental.preprocessing import StringLookup
from tensorflow import keras

import matplotlib.pyplot as plt
import tensorflow as tf
import numpy as np
import pandas as pd
import os

In [9]:
from sklearn.model_selection import train_test_split

In [10]:
def distortion_free_resize(image, img_size): #maybe no need for this
    w, h = img_size
    image = tf.image.resize(image, size=(h, w), preserve_aspect_ratio=True)

    # Check tha amount of padding needed to be done.
    pad_height = h - tf.shape(image)[0]
    pad_width = w - tf.shape(image)[1]

    # Only necessary if you want to do same amount of padding on both sides.
    if pad_height % 2 != 0:
        height = pad_height // 2
        pad_height_top = height + 1
        pad_height_bottom = height
    else:
        pad_height_top = pad_height_bottom = pad_height // 2

    if pad_width % 2 != 0:
        width = pad_width // 2
        pad_width_left = width + 1
        pad_width_right = width
    else:
        pad_width_left = pad_width_right = pad_width // 2

    image = tf.pad(
        image,
        paddings=[
            [pad_height_top, pad_height_bottom],
            [pad_width_left, pad_width_right],
            [0, 0],
        ],
    )

    image = tf.transpose(image, perm=[1, 0, 2])
    image = tf.image.flip_left_right(image)
    return image

In [40]:
AUTOTUNE = tf.data.AUTOTUNE

max_len = 94 #!!!To trzeba zmienić!!!
padding_token = 99
batch_size = 64
image_width = 512
image_height = 64
# Mapping characters to integers.
char_to_num = StringLookup(vocabulary=list(dictionary), mask_token=None)

# Mapping integers back to original characters.
num_to_char = StringLookup(
    vocabulary=char_to_num.get_vocabulary(), mask_token=None, invert=True
)


def preprocess_image(image_path, img_size=(image_width, image_height)):
    image = tf.io.read_file(image_path)
    image = tf.image.decode_png(image, 1)
    image = distortion_free_resize(image, img_size)
    image = tf.cast(image, tf.float32) / 255.0
    return image


def vectorize_label(label):
    label = char_to_num(tf.strings.unicode_split(label, input_encoding="UTF-8"))
    length = tf.shape(label)[0]
    pad_amount = max_len - length
    label = tf.pad(label, paddings=[[0, pad_amount]], constant_values=padding_token)
    return label


def process_images_labels(image_path, label):
    image = preprocess_image(image_path)
    label = vectorize_label(label)
    return {"image": image, "label": label}


def prepare_dataset(image_paths, labels):
    dataset = tf.data.Dataset.from_tensor_slices((image_paths, labels)).map(
        process_images_labels, num_parallel_calls=AUTOTUNE
    )
    return dataset.batch(batch_size).cache().prefetch(AUTOTUNE)


In [41]:
data = pd.read_csv("test\\test.csv").drop(columns=["Unnamed: 0"], index=62)
data["path"] = [f"test\\{i+1}.png" for i in range(data.shape[0])]

In [42]:
# the neural net needs a list of strings as labels so we need to change this back...


In [43]:
X_train, X_test_val, y_train, y_test_val = train_test_split(
    ['x'.join(x) for x in data.drop(columns=['path']).values.tolist()], 
    data["path"],
    test_size=0.2,
)

X_test, X_val, y_test, y_val = train_test_split(
    X_test_val,
    y_test_val,
    test_size = 0.5
)

train_ds = prepare_dataset(y_train, X_train)
validation_ds = prepare_dataset(y_val, X_val)
test_ds = prepare_dataset(y_test, X_test)

validation_images = []
validation_labels = []

for batch in validation_ds:
    validation_images.append(batch["image"])
    validation_labels.append(batch["label"])

In [44]:
class CTCLayer(keras.layers.Layer):
    def __init__(self, name=None):
        super().__init__(name=name)
        self.loss_fn = keras.backend.ctc_batch_cost

    def call(self, y_true, y_pred):
        batch_len = tf.cast(tf.shape(y_true)[0], dtype="int64")
        input_length = tf.cast(tf.shape(y_pred)[1], dtype="int64")
        label_length = tf.cast(tf.shape(y_true)[1], dtype="int64")

        input_length = input_length * tf.ones(shape=(batch_len, 1), dtype="int64")
        label_length = label_length * tf.ones(shape=(batch_len, 1), dtype="int64")
        loss = self.loss_fn(y_true, y_pred, input_length, label_length, )
        self.add_loss(loss)

        # At test time, just return the computed predictions.
        return y_pred



def build_model():
    # Inputs to the model
    input_img = keras.Input(shape=(image_width, image_height, 1), name="image")
    labels = keras.layers.Input(name="label", shape=(None,))

    # First conv block.
    x = keras.layers.Conv2D(
        32,
        (3, 3),
        activation="relu",
        kernel_initializer="he_normal",
        padding="same",
        name="Conv1",
    )(input_img)
    x = keras.layers.MaxPooling2D((4, 4), name="pool1")(x)

    # Second conv block.
    x = keras.layers.Conv2D(
        32,
        (3, 3),
        activation="relu",
        kernel_initializer="he_normal",
        padding="same",
        name="Conv2",
    )(x)
    #x = keras.layers.MaxPooling2D((2, 2), name="pool2")(x)

    # We have used two max pool with pool size and strides 2.
    # Hence, downsampled feature maps are 4x smaller. The number of
    # filters in the last layer is 64. Reshape accordingly before
    # passing the output to the RNN part of the model.
    new_shape = ((image_width // 4), (image_height // 4) * 32)
    x = keras.layers.Reshape(target_shape=new_shape, name="reshape")(x)
    x = keras.layers.Dense(32, activation="relu", name="dense1")(x)
    x = keras.layers.Dropout(0.2)(x)

    # RNNs.
    x = keras.layers.Bidirectional(
        keras.layers.LSTM(64, return_sequences=True, dropout=0.25)
    )(x)
    x = keras.layers.Bidirectional(
        keras.layers.LSTM(32, return_sequences=True, dropout=0.25)
    )(x)

    # +2 is to account for the two special tokens introduced by the CTC loss.
    # The recommendation comes here: https://git.io/J0eXP.
    x = keras.layers.Dense(
        len(char_to_num.get_vocabulary()) + 2, activation="softmax", name="dense2"
    )(x)

    # Add CTC layer for calculating CTC loss at each step.
    output = CTCLayer(name="ctc_loss")(labels, x)

    # Define the model.
    model = keras.models.Model(
        inputs=[input_img, labels], outputs=output, name="handwriting_recognizer"
    )
    # Optimizer.
    opt = keras.optimizers.Adam()
    # Compile the model and return.
    model.compile(optimizer=opt)
    return model


# Get the model.
model = build_model()
model.summary()

Model: "handwriting_recognizer"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 image (InputLayer)             [(None, 512, 64, 1)  0           []                               
                                ]                                                                 
                                                                                                  
 Conv1 (Conv2D)                 (None, 512, 64, 32)  320         ['image[0][0]']                  
                                                                                                  
 pool1 (MaxPooling2D)           (None, 128, 16, 32)  0           ['Conv1[0][0]']                  
                                                                                                  
 Conv2 (Conv2D)                 (None, 128, 16, 32)  9248        ['pool1[0][0

In [45]:
def calculate_edit_distance(labels, predictions):
    # Get a single batch and convert its labels to sparse tensors.
    saprse_labels = tf.cast(tf.sparse.from_dense(labels), dtype=tf.int64)

    # Make predictions and convert them to sparse tensors.
    input_len = np.ones(predictions.shape[0]) * predictions.shape[1]
    predictions_decoded = keras.backend.ctc_decode(
        predictions, input_length=input_len, greedy=True
    )[0][0][:, :max_len]
    sparse_predictions = tf.cast(
        tf.sparse.from_dense(predictions_decoded), dtype=tf.int64
    )

    # Compute individual edit distances and average them out.
    edit_distances = tf.edit_distance(
        sparse_predictions, saprse_labels, normalize=False
    )
    return tf.reduce_mean(edit_distances)


class EditDistanceCallback(keras.callbacks.Callback):
    def __init__(self, pred_model):
        super().__init__()
        self.prediction_model = pred_model

    def on_epoch_end(self, epoch, logs=None):
        edit_distances = []

        for i in range(len(validation_images)):
            labels = validation_labels[i]
            predictions = self.prediction_model.predict(validation_images[i])
            edit_distances.append(calculate_edit_distance(labels, predictions).numpy())

        print(
            f"Mean edit distance for epoch {epoch + 1}: {np.mean(edit_distances):.4f}")

In [46]:
epochs = 50  # To get good results this should be at least 50.

model = build_model()
prediction_model = keras.models.Model(
    model.get_layer(name="image").input, model.get_layer(name="dense2").output
)
edit_distance_callback = EditDistanceCallback(prediction_model)

# Train the model.
history = model.fit(
    train_ds,
    validation_data=test_ds,
    epochs=epochs,
    callbacks=[edit_distance_callback],
)

Epoch 1/50


InvalidArgumentError: Graph execution error:

Paddings must be non-negative: 0 -2
	 [[{{node PadV2}}]]
	 [[IteratorGetNext]] [Op:__inference_train_function_45206]

In [ ]:
def decode_batch_predictions(pred):
    input_len = np.ones(pred.shape[0]) * pred.shape[1]
    # Use greedy search. For complex tasks, you can use beam search.
    results = keras.backend.ctc_decode(pred, input_length=input_len, greedy=True)[0][0][
        :, :max_len
    ]
    # Iterate over the results and get back the text.
    output_text = []
    for res in results:
        res = tf.gather(res, tf.where(tf.math.not_equal(res, -1)))
        res = tf.strings.reduce_join(num_to_char(res)).numpy()#.decode("utf-8")
        output_text.append(res)
    return output_text


#  Let's check results on some test samples.
for batch in test_ds.take(1):
    batch_images = batch["image"]
    _, ax = plt.subplots(4, 1, figsize=(15, 8))

    preds = prediction_model.predict(batch_images)
    pred_texts = decode_batch_predictions(preds)

    for i in range(4):
        img = batch_images[i]
        img = tf.image.flip_left_right(img)
        img = tf.transpose(img, perm=[1, 0, 2])
        img = (img * 255.0).numpy().clip(0, 255).astype(np.uint8)
        img = img[:, :, 0]

        title = f"Prediction: {pred_texts[i]}"
        ax[i // 4].imshow(img, cmap="gray")
        ax[i // 4].set_title(title)
        ax[i // 4].axis("off")